In [1]:
using Plots
using PlotlyJS
using Statistics
using QuadGK
using MAT

In [18]:
vars_q4 = matread("../solutions/Q4data.mat")
time = vars_q4["time"];
Tout = vars_q4["Tout"];
TintRec = vars_q4["TintMeas"];

In [3]:
# definining constants again, just for completeness and avoid issues..
va      = 2880;     # Volume of air inside the building [m^3]
rhoa    = 1.225;     # Density of air [kg/m^3]
ca      = 1005;      # Specific heat of air [J/KgK]
th      = 0.1;       # Thickness of thermal mass floors[m]
A_f     = 2*90;       # Area of the 3 floors [m^2]
rho_f   = 2300;     # Density of Thermal Mass concrete [kg/m^3]
k_f     = 0.8;         # Conductivity of Thermal Mass concrete [W/mK]
c_f     = 750;        # Specific heat of Thermal Mass concrete [J/KgK]
hconv   = 4;           # Convective heat coefficient [W/m2.K]
Eint    = 250;      # Internal loads due to occupancy, lighting and equipment [W]

In [ ]:
# do we have to implement own version of E_v or not? 

## Fabric calculation 

In [5]:
# Discretization variables
dt = 15;
dx = th/10;
x = range(0, th, step = dx)  |> collect 
Nx = length(x)

11

In [14]:
function conductionMatrix(lambda, dt, dx, Nx)
    # used to calculate temperature evolution within a fabric 
    A = zeros(Nx, Nx)

    A[1,1] = 1-lambda;  
    A[1,2] = lambda;
    A[Nx,Nx-1] = lambda;
    A[Nx,Nx] = 1-lambda;  
    
    # all other rows 
    for i = 2:Nx-1
        A[i, i-1] = lambda;
        A[i, i] = 1-2*lambda;
        A[i, i+1] = lambda;
    end

    return A

end

conductionMatrix (generic function with 1 method)

In [15]:
# Matrix for conduction 
alpha = k_f/(rho_f*c_f) # thermal conductivity / (density * specific heat )
lambda = alpha * dt / dx^2
A = conductionMatrix(lambda, dt, dx, Nx);


In [21]:
# first loop for thermal mass initial temperature 

# initialize night flush conditions => convert from clecius to kelvin 
Tint    = 273+TintRec[1]; 
T0      = 273+TintRec[1];
TNx     = 273+TintRec[1];
Tf      = T0 .+ (TNx-T0)*x'/th; # !!! why not just set the fabric temp to T0?? => 1x11 matrix of 296...

## natural ventilation 

In [32]:
g = 9.81;

In [33]:
function cd_pivot(alpha)
    W=1;
    H=1;
    W_pivot = z -> (1/W^2 + 1/(2*(H-z)*tand(alpha)+sind(alpha)*W).^2) .^(-1/2)
    h = H * (1- cosd(alpha));
    integral, est = quadgk(W_pivot,h,H) #rtol=1e-8 -> error when go to last element of angle array
    A_eff = W*h + integral ;
    Cd0 = 0.611;
    Cd = A_eff / (H*W) * Cd0;   
    
end

cd_pivot (generic function with 1 method)

### windows

In [ ]:
# window properties 
w1 = Dict("alpha" => 42, "area" => 1.61, "l" => 11.34)
w1["cd"] = cd_pivot(w1["alpha"])

w2 = Dict("alpha" => 42, "area" => 1.755, "l" => 6.62)
w2["cd"] = cd_pivot(w2["alpha"])

w3 = Dict("alpha" => 42, "area" => 1.755, "l" => 2.07)
w3["cd"] = cd_pivot(w3["alpha"]);

In [38]:
# convert data into kelvin 
time_in_seconds = time*3600;
time_in_hours   = time;
Tout        = 273 .+Tout;
TintReal    = 273 .+TintRec;

## recursive loop - compute ventilation, internal temperature, and fabric temperature..

In [ ]:
for t=1:length(time)
    # equivalent hour 
    h = time_in_hours[t];


    # calculate energy of nat vent at each time step... 
    # TODO..?

    # Q = \dot{V} => rate of change of volume 
    for (w, Q) in zip([w1, w2, w3], [Q1, Q2, Q3])
        try 
            Q[t] = w["cd"] * w["area"] * sqrt(2*g*w["l"] *(TintMeas[i]-Tout[i])/(Tout[i]+273));
        catch y
            if isa(y, DomainError)
                # windows should be closed if Tint < Tout, which is what would cause a domain error 
                Q[i] = 0
            end
        end 
    end
    Q_tot[t] = Q1[t] + Q2[t] + Q3[t]
    Env[t] = rhoa * ca * Q_tot[t] * (Tout[t] - Tint[t])


    # convective heat flux
    qconv = hconv* (Tf[1,t] - Tint[t]) # first floor
    qconvst[t] = qconv

    # wall boundary conditions 
    # at x = 0 -->
    # TODO
    b[1] = - lambda * dx / (k_f * qconv)
    b[Nx] = - lambda * dx / (k_f * qconv)




 end